<a href="https://colab.research.google.com/github/rpasquini/metodos_cuantitativos/blob/main/notebooks/R/OLS_2_Ajuste_Test_Hipotesis_V2_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

_author: Ricardo Pasquini (rpasquini@gmail.com)


In [ ]:
%load_ext rpy2.ipython


# OLS: Ajuste, Propiedades y Test de Hipótesis

En este notebook vamos a explorar:
1. Cómo ajustar un modelo de regresión lineal simple
2. Las propiedades del estimador OLS
3. Tests de hipótesis sobre los coeficientes


In [ ]:
%%R

# Instalamos los paquetes necesarios si no están instalados
if (!require("sf")) install.packages("sf")
if (!require("dplyr")) install.packages("dplyr")
if (!require("ggplot2")) install.packages("ggplot2")
if (!require("car")) install.packages("car")  # Para diagnósticos de regresión
if (!require("lmtest")) install.packages("lmtest")  # Para pruebas de hipótesis
if (!require("sandwich")) install.packages("sandwich")  # Para errores estándar robustos


In [ ]:
%%R

# Cargamos las librerías
library(sf)
library(dplyr)
library(ggplot2)
library(car)
library(lmtest)
library(sandwich)


In [ ]:
%%R

# Descargamos y cargamos los datos
download.file("https://github.com/rpasquini/urban-econometrics/blob/master/data/departamentos2019amba.zip?raw=true",
              destfile = "departamentos2019amba.zip", mode = "wb")
unzip("departamentos2019amba.zip", exdir = ".")
departamentos <- st_read("departamentos2019amba.shp")


# Estimación del modelo lineal


Comenzaremos por replicar la estimación de los coeficientes del modelo:

$price_i=\beta_0+\beta_1*bedrooms_i+\epsilon_i$


In [ ]:
%%R

# Ajustamos el modelo usando bedrooms como variable independiente
modelo <- lm(price ~ bedrooms, data = departamentos)

# Resumen del modelo
summary(modelo)


# Predicción

$\hat{Y}=\hat{\beta_0}+\hat{\beta_1}*X$


In [ ]:
%%R

# Departamento de 2 dormitorios
coef(modelo)[1] + coef(modelo)[2] * 2


In [ ]:
%%R

# Predicciones para todas las observaciones en la data
departamentos_in_regression <- departamentos[!is.na(departamentos$bedrooms) & !is.na(departamentos$price), ]
predicciones <- predict(modelo)
head(predicciones)


In [ ]:
%%R

# Ploteamos predicción con ggplot2
ggplot(departamentos_in_regression, aes(x = bedrooms, y = predicciones)) +
  geom_point() +
  labs(x = "Dormitorios", y = "Precio predicho",
       title = "Predicciones vs. Número de dormitorios") +
  theme_minimal()


# Error Cuadrático Medio

Recordemos que la definición del error cuadrático medio es:

$MSE=\sqrt{\frac{\sum^n_{i=1} e_i^2}{n-2}}$ donde

$e=y-\hat{y}$


In [ ]:
%%R

# Calculamos MSE
mse <- sum(residuals(modelo)^2) / (length(residuals(modelo)) - 2)
mse


In [ ]:
%%R

# Raíz cuadrada del MSE
sqrt(mse)


# ¿Cómo son los residuos (errores estimados) del modelo estimado?

## Primero analizo la normalidad de la distribución
Inspecciono el histograma para tener una idea rápida de la distribución


In [ ]:
%%R

# Histograma de residuos
ggplot(data.frame(residuos = residuals(modelo)), aes(x = residuos)) +
  geom_histogram(bins = 50) +
  labs(title = "Distribución de los residuos",
       x = "Residuos",
       y = "Frecuencia") +
  theme_minimal()


## Analizamos visualmente si la varianza del residuo es constante (homocedasticidad)


In [ ]:
%%R

# Agregamos los residuos al dataframe
departamentos_in_regression$residuos <- residuals(modelo)

# Gráfico de residuos vs. variable independiente
ggplot(departamentos_in_regression, aes(x = bedrooms, y = residuos)) +
  geom_point() +
  geom_hline(yintercept = 0, linetype = "dashed", color = "red") +
  labs(title = "Residuos vs. Número de dormitorios",
       x = "Dormitorios",
       y = "Residuos") +
  theme_minimal()


## Analizamos visualmente la distribución espacial en busca de correlación espacial
(más adelante implementaremos medidas específicas)


In [ ]:
%%R

# Mapa de residuos
ggplot() +
  geom_sf(data = departamentos_in_regression, aes(color = residuos)) +
  scale_color_gradient2(low = "blue", mid = "white", high = "red", midpoint = 0) +
  labs(title = "Distribución espacial de residuos",
       color = "Residuos") +
  theme_minimal()


### Agregando capa de referencia con mapas base
Para agregar un mapa base en R, podemos usar el paquete `mapview` o `leaflet`. Sin embargo, como estamos usando Colab y rpy2, mantendremos el mapa simple por ahora.


# Exploración inicial de los datos

Vamos a explorar la relación entre el precio de los departamentos y su superficie.


In [ ]:
%%R

# Gráfico de dispersión
ggplot(departamentos, aes(x = surface_to, y = price)) +
  geom_point(alpha = 0.5) +
  geom_smooth(method = "lm", color = "red") +
  labs(x = "Superficie (m²)", y = "Precio", title = "Relación entre Precio y Superficie") +
  theme_minimal()
